<a href="https://colab.research.google.com/github/adeakinwe/ML-Google-Colab-Notes/blob/main/DNNClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

In [ ]:
COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']

In [ ]:
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [ ]:
train_path = tf.keras.utils.get_file("iris_training.csv", "http://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "http://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
#keras is a tensorflow module

train = pd.read_csv(train_path, names=COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=COLUMN_NAMES, header=0)

In [ ]:
train.head()
#test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


In [ ]:
tf.shape(train)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([120,   5], dtype=int32)>

In [ ]:
tf.shape(test)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([30,  5], dtype=int32)>

In [ ]:
y_train = train.pop('Species')

In [ ]:
y_test = test.pop('Species')

In [ ]:
def input_fn(features,labels,training=True,batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    hidden_units = [30, 10],
    n_classes = 3
)

classifier.train(
    input_fn = lambda: input_fn(train,y_train,training=True),
    steps=5000
)

In [ ]:
test_result = classifier.evaluate(input_fn = lambda: input_fn(test,y_test,training=False))
print(test_result)
print('\nTest accuracy: {accuracy:0.3f}\n'.format(**test_result))

{'accuracy': 0.76666665, 'average_loss': 0.7216273, 'loss': 0.7216273, 'global_step': 5000}

Test accuracy: 0.767



# New Section

#Taking inputs and predicting the specie

In [ ]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print('please type numeric values as prompted')
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn= lambda: input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('prediction is "{}" ({:.1f}%)'.format(
      SPECIES[class_id], 100*probability
      ))



please type numeric values as prompted
SepalLength: 2.3
SepalWidth: 6.3
PetalLength: 2.3
PetalWidth: 4.1
{'logits': array([ 0.08818629, -0.03914052, -1.2549603 ], dtype=float32), 'probabilities': array([0.46696916, 0.4111411 , 0.12188971], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
prediction is "Setosa" (46.7%)


Exception ignored in: <generator object Estimator.predict at 0x7f8d9d848bd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow_estimator/python/estimator/estimator.py", line 652, in predict
  File "/usr/lib/python3.7/contextlib.py", line 130, in __exit__
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 5827, in get_controller
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/context.py", line 302, in pop
IndexError: pop from empty list
